In [4]:
import pandas as pd
# Import required libraries and dependencies

import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error



# Import Modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [5]:
attributes_df = pd.read_csv("../Resources/songAttributes_1999-2019.csv")


billboard_df = pd.read_csv("../Resources/billboardHot100_1999-2019.csv")


artists_df = pd.read_csv("../Resources/artistDf.csv")


weekly_df = pd.read_csv("../Resources/spotifyWeeklyTop200Streams.csv")

weekly_df

FileNotFoundError: [Errno 2] No such file or directory: '../Resources/billboardHot100_1999-2019.csv'

In [ ]:
weekly_df

,Unnamed: 0,Name,Artist,Features,Streams,Week
0,0,In My Feelings,Drake,NaN,30747676,2018-07-20
1,1,Lucid Dreams,Juice WRLD,NaN,12930705,2018-07-20
2,2,Nonstop,Drake,NaN,12312859,2018-07-20
3,3,God is a woman,Ariana Grande,NaN,10771324,2018-07-20
4,4,SAD!,XXXTENTACION,NaN,10503061,2018-07-20
...,...,...,...,...,...,...
26390,26390,R.I.P. SCREW,Travis Scott,NaN,1521806,2018-09-14
26391,26391,Rich & Sad,Post Malone,NaN,1518345,2018-09-14
26392,26392,The Way Life Goes,Lil Uzi Vert,Oh Wonder,1516966,2018-09-14
26393,26393,broken,lovelytheband,NaN,1486767,2018-09-14


In [ ]:
weekly_df.drop(columns=["Unnamed: 0", "Name"], inplace=True)

In [ ]:
grouped = weekly_df.groupby("Artist")["Streams"].sum()
grouped_reset = grouped.reset_index()
grouped_dict = grouped_reset.to_dict(orient="split")
# total_streams_df = pd.DataFrame(grouped_dict)


# Create a DataFrame from the dictionary
total_streams_df = pd.DataFrame(grouped_dict['data'], columns=grouped_dict['columns'])

# Display the resulting DataFrame
total_streams_df

,Artist,Streams
0,*NSYNC,5690510
1,2 Chainz,275432430
2,21 Savage,914888503
3,22 Savage,4982496
4,5 Seconds of Summer,236805582
...,...,...
590,"benny blanco, Jesse, Swae Lee, Jesse, Swae Lee",1808602
591,"benny blanco, Juice WRLD feat. Brendon Urie, J...",50309105
592,blackbear,152112104
593,gnash,18720111


In [ ]:
weekly_df = weekly_df.drop_duplicates(subset = "Artist", keep = "first").reset_index()

In [ ]:
weekly_df.drop(columns=["index"], inplace=True)

In [ ]:
weekly_df

,Artist,Features,Streams,Week
0,Drake,NaN,30747676,2018-07-20
1,Juice WRLD,NaN,12930705,2018-07-20
2,Ariana Grande,NaN,10771324,2018-07-20
3,XXXTENTACION,NaN,10503061,2018-07-20
4,Tyga,Offset,9777479,2018-07-20
...,...,...,...,...
590,The xx,NaN,1153955,2017-01-20
591,WizKid,NaN,1242932,2017-04-07
592,Tame Impala,NaN,1633712,2019-04-19
593,Mitchell Tenpenny,NaN,1462815,2019-01-11


In [ ]:
final_df = weekly_df.merge(total_streams_df, how="inner", on="Artist")
final_df

,Artist,Features,Streams_x,Week,Streams_y
0,Drake,NaN,30747676,2018-07-20,3441947366
1,Juice WRLD,NaN,12930705,2018-07-20,1445959305
2,Ariana Grande,NaN,10771324,2018-07-20,1598012598
3,XXXTENTACION,NaN,10503061,2018-07-20,2872157309
4,Tyga,Offset,9777479,2018-07-20,313734165
...,...,...,...,...,...
590,The xx,NaN,1153955,2017-01-20,2220061
591,WizKid,NaN,1242932,2017-04-07,1242932
592,Tame Impala,NaN,1633712,2019-04-19,4011775
593,Mitchell Tenpenny,NaN,1462815,2019-01-11,1462815


In [ ]:
final_df = final_df.drop(columns=["Streams_x"])
final_df

,Artist,Features,Week,Streams_y
0,Drake,NaN,2018-07-20,3441947366
1,Juice WRLD,NaN,2018-07-20,1445959305
2,Ariana Grande,NaN,2018-07-20,1598012598
3,XXXTENTACION,NaN,2018-07-20,2872157309
4,Tyga,Offset,2018-07-20,313734165
...,...,...,...,...
590,The xx,NaN,2017-01-20,2220061
591,WizKid,NaN,2017-04-07,1242932
592,Tame Impala,NaN,2019-04-19,4011775
593,Mitchell Tenpenny,NaN,2019-01-11,1462815


In [ ]:
final_df.rename(columns={"Streams_y":"Total streams"}, inplace=True)

In [ ]:
artists_df["Artist"] = artists_df["Artist"].str.lower()
final_df["Artist"] = final_df["Artist"].str.lower()

In [ ]:
artist_week = artists_df.merge(final_df, how="outer", on="Artist")

In [ ]:
artist_week["Total streams"].notna().value_counts()

False    905
True     595
Name: Total streams, dtype: int64

In [ ]:
artist_week = artist_week.dropna(subset = ["Total streams"])

In [ ]:
artist_week.drop(columns=["X"], inplace=True)

In [ ]:
artist_week.reset_index(inplace=True)
artist_week.drop(columns=["index"])

,Artist,Followers,Genres,NumAlbums,YearFirstAlbum,Gender,Group.Solo,Features,Week,Total streams
0,ed sheeran,52698756.0,"pop,uk pop",8.0,2011.0,M,Solo,NaN,2018-07-20,9.361553e+08
1,justin bieber,30711450.0,"canadian pop,dance pop,pop,post-teen pop",10.0,2009.0,M,Solo,NaN,2018-12-14,4.543449e+07
2,jonas brothers,3069527.0,"boy band,dance pop,pop,post-teen pop",10.0,2006.0,M,Group,NaN,2019-04-12,1.825584e+08
3,drake,41420478.0,"canadian hip hop,canadian pop,hip hop,pop rap,...",11.0,2010.0,M,Solo,NaN,2018-07-20,3.441947e+09
4,chris brown,9676862.0,"dance pop,pop,pop rap,r&b,rap",6.0,2005.0,M,Solo,NaN,2017-11-10,1.383342e+08
...,...,...,...,...,...,...,...,...,...,...
590,the xx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-01-20,2.220061e+06
591,wizkid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-04-07,1.242932e+06
592,tame impala,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-19,4.011775e+06
593,mitchell tenpenny,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-11,1.462815e+06


In [ ]:
artist_week.to_csv("../Resources/ArtistWeek.csv")

In [ ]:
final_df

,Artist,Features,Week,Total streams
0,drake,NaN,2018-07-20,3441947366
1,juice wrld,NaN,2018-07-20,1445959305
2,ariana grande,NaN,2018-07-20,1598012598
3,xxxtentacion,NaN,2018-07-20,2872157309
4,tyga,Offset,2018-07-20,313734165
...,...,...,...,...
590,the xx,NaN,2017-01-20,2220061
591,wizkid,NaN,2017-04-07,1242932
592,tame impala,NaN,2019-04-19,4011775
593,mitchell tenpenny,NaN,2019-01-11,1462815


In [7]:
attributes_df = pd.read_csv("../Resources/songAttributes_1999-2019.csv")


billboard_df = pd.read_csv("../Resources/billboardHot100_1999-2019.csv")

In [13]:
# billboard_df.drop(columns=["Unnamed: 0"], inplace=True)
# attributes_df.drop(columns=["Unnamed: 0"], inplace=True)

In [18]:
billboard_df["Artists"] = billboard_df["Artists"].str.lower()
billboard_df["Name"] = billboard_df["Name"].str.lower()

attributes_df["Artist"] = attributes_df["Artist"].str.lower()
attributes_df["Name"] = attributes_df["Name"].str.lower()

In [19]:
billboard_df.rename(columns={"Artists":"Artist"}, inplace=True)

In [21]:
billboard_attributes_df = billboard_df.merge(attributes_df, on=["Name", "Artist"], how ="inner")

In [24]:
billboard_df

,Artist,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features
0,"lil nas,",old town road,1,1.0,7.0,2019-07-06,"April 5, 2019","Country,Atlanta,Alternative Country,Hip-Hop,Tr...","Jozzy, Atticus ross, Trent reznor, Billy ray c...","Old Town Road Remix \nOh, oh-oh\nOh\nYeah, I'm...",Billy Ray Cyrus
1,"shawn mendes, camila cabello",senorita,2,NaN,NaN,2019-07-06,"June 21, 2019",Pop,"Cashmere cat, Jack patterson, Charli xcx, Benn...",Senorita \nI love it when you call me senorita...,NaN
2,billie eilish,bad guy,3,2.0,13.0,2019-07-06,"March 29, 2019","Hip-Hop,Dark Pop,House,Trap,Memes,Alternative ...","Billie eilish, Finneas","bad guy \nWhite shirt now red, my bloody nose\...",NaN
3,khalid,talk,4,3.0,20.0,2019-07-06,"February 7, 2019","Synth-Pop,Pop","Howard lawrence, Guy lawrence, Khalid",Talk \nCan we just talk? Can we just talk?\nTa...,NaN
4,"ed sheeran, justin bieber",i don't care,5,2.0,7.0,2019-07-06,"May 10, 2019","Canada,UK,Dance,Dance-Pop,Pop","Ed sheeran, Justin bieber, Shellback, Max mart...",I Don't Care \nI'm at a party I don't wanna be...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
97220,vitamin c,smile,95,NaN,NaN,1999-07-12,NaN,"Jamaica,Pop","Colleen fitzpatrick, Josh deutsch","Smile \nHahaha\nAlright, yeah\nAlright\nFirst ...",Lady Saw
97221,collective soul,heavy,96,73.0,20.0,1999-07-12,NaN,"Hockey,Gaming,Soundtrack,Rock",Collective soul,Heavy \nComplicate this world you wrapped for ...,NaN
97222,mary chapin carpenter,almost home,97,NaN,NaN,1999-07-12,NaN,"Country,Pop","Annie roboff, Beth nielsen chapman, Mary chapi...",Almost Home \nI saw my life this morning\nLyin...,NaN
97223,q,vivrant thing,98,NaN,NaN,1999-07-12,NaN,Rap,"Q tip, J dilla, Barry white",Vivrant Thing \nUh check it out now\nUh no dou...,NaN


In [26]:
billboard_attributes_df.corr()

,Weekly.rank,Peak.position,Weeks.on.chart,Acousticness,Danceability,Duration,Energy,Explicit,Instrumentalness,Liveness,Loudness,Mode,Popularity,Speechiness,Tempo,TimeSignature,Valence
Weekly.rank,1.000000,0.816136,-0.296250,0.025789,-0.168166,-0.019392,0.019893,-0.028965,0.012518,0.046944,-0.020136,0.112484,-0.109483,-0.055065,0.051767,-0.033998,-0.081051
Peak.position,0.816136,1.000000,-0.544164,0.036963,-0.190119,-0.029671,0.016968,-0.051719,0.016436,0.031819,-0.033186,0.127173,-0.132918,-0.082270,0.056869,-0.040388,-0.080067
Weeks.on.chart,-0.296250,-0.544164,1.000000,0.021974,0.051400,-0.002311,-0.039223,-0.051238,0.067546,-0.048138,-0.006043,-0.040755,0.082689,-0.047950,-0.016389,0.002568,-0.006297
Acousticness,0.025789,0.036963,0.021974,1.000000,-0.059029,0.017026,-0.546466,-0.063464,0.034332,-0.058719,-0.409551,0.081551,-0.066489,-0.045020,-0.108545,-0.064645,-0.171024
Danceability,-0.168166,-0.190119,0.051400,-0.059029,1.000000,-0.046560,-0.060799,0.250027,0.015118,-0.124293,-0.019831,-0.142119,0.033381,0.199279,-0.271118,0.111586,0.440292
Duration,-0.019392,-0.029671,-0.002311,0.017026,-0.046560,1.000000,-0.134598,0.100470,0.011759,0.047743,-0.113493,-0.040244,0.015956,0.088763,-0.055778,-0.019667,-0.222535
Energy,0.019893,0.016968,-0.039223,-0.546466,-0.060799,-0.134598,1.000000,-0.038155,-0.094851,0.155258,0.712533,-0.057574,0.024073,0.032677,0.147034,0.095632,0.388338
Explicit,-0.028965,-0.051719,-0.051238,-0.063464,0.250027,0.100470,-0.038155,1.000000,-0.046591,0.067190,-0.010311,-0.116584,0.098582,0.486186,-0.040634,0.066912,0.016991
Instrumentalness,0.012518,0.016436,0.067546,0.034332,0.015118,0.011759,-0.094851,-0.046591,1.000000,-0.013855,-0.161183,0.027940,-0.025920,-0.051965,-0.008212,-0.005706,-0.068310
Liveness,0.046944,0.031819,-0.048138,-0.058719,-0.124293,0.047743,0.155258,0.067190,-0.013855,1.000000,0.065940,0.013063,-0.049718,0.125967,0.040389,-0.002096,0.011735
